# Goal

* Create a planning react agent
* Tools
  * `planner`
    * chain-of-thought
  * `critic`
  * `router`

In [1]:
# import 
import os
import time
from enum import Enum
from pprint import pprint
from typing import Annotated, List, Dict, Tuple, Optional, Union, Any
from pydantic import BaseModel, Field
from langchain_core.tools import tool
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

In [2]:
# setup
load_dotenv()
pd.set_option('display.max_colwidth', 1000)
os.environ["DEBUG_MODE"] = "TRUE"

# Tools

## Planner

In [3]:
model_t2 = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
model_t0 = ChatOpenAI(model="gpt-4o", temperature=0)

In [4]:
class Step(BaseModel):
    explanation: str
    output: str

class Planning(BaseModel):
    steps: list[Step]

@tool
def invoke_planner(
    message: Annotated[str, "Message to the planner"],
) -> Annotated[str, "Response from the planner"]:
    """
    The planner will help you plan how to accomplish a task.
    Be specific about the task you need help with.
    """
    prompt = "\n".join([
        "You are an expert planner.",
        "Think step by step to help the user accomplish the task.",
        "The task: ",
        message
    ])
    response = model_t2.with_structured_output(Planning, strict=True).invoke(prompt)
    steps = "\n".join([f" - {x.output}" for x in response.steps])
    return {
        "messages": [HumanMessage(content=steps, name="planner")]
    }

# invoke_planner.invoke({"message" : "How do I convert GSE121737 to an SRA accession?"})

## Critic

In [5]:
class Reasons(BaseModel):
    explanation: str
    output: str

class Critque(BaseModel):
    critique: str
    reasoning: list[Reasons]

@tool
def invoke_critic(
    message: Annotated[str, "Message to the critic"],
) -> Annotated[str, "Response from the critic"]:
    """
    The critic will help you critique a plan to accomplish a task.
    """
    prompt = "\n".join([
        "You are an expert critic.",
        "Think step by step to critique the plan.",
        message
    ])
    response = model_t2.with_structured_output(Critque, strict=True).invoke(prompt)
    reasoning = "\n".join([f" - {x.output}" for x in response.reasoning])
    reasoning = f"{response.critique}\nMy reasoning:\n{reasoning}"
    return {
        "messages": [HumanMessage(content=reasoning, name="critic")]
    }

In [6]:
supervisor_agent = create_react_agent(
    model=model_t0,
    tools=[invoke_planner, invoke_critic],
    state_modifier="\n".join([
        "You are a supervisor.",
        "Your job is to help the user plan and critique a plan to accomplish a task.",
        "Facilitate the conversation between the planner and the critic.",
        "When the planner is done, ask the critic to critique the plan.",
        "When the critic is done, ask the planner to revise the plan.",
        "Use up to 3 planner-critic rounds to accomplish the task.",
        "Return the final plan.",
    ])
)

# invoke the agent
inputs = {"messages": [("user", "How do I convert GSE121737 to an SRA accession using Bio.Entrez?")]}
config = {"max_concurrency" : 3, "recursion_limit": 30}
for step in supervisor_agent.stream(inputs, config=config):
    print(step)


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MDSBuEZHpecOgY4HNrSCtMlZ', 'function': {'arguments': '{"message":"Plan a step-by-step guide to convert GSE121737 to an SRA accession using Bio.Entrez in Python."}', 'name': 'invoke_planner'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 200, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_7f6be3efb0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-089ce842-5fcf-40e7-a240-db69fa3d723a-0', tool_calls=[{'name': 'invoke_planner', 'args': {'message': 'Plan a step-by-step guide to convert GSE121737 to an SRA accession using Bio.Entrez in Python.'}, 'id': 'call_MDSBuEZHpecOgY4HNrSCtMl